In [2]:
import pandas as pd
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

In [3]:
model = load_model('model.h5')

In [4]:
with open('onehot_encoder_geo.pkl' , 'rb') as f:
    onehot_encoder_geo = pickle.load(f)

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [10]:
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':42,
    'Tenure':7,
    'Balance':84000,
    'NumOfProducts':1,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [11]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Dell\python\PROJECT\MLOPS\Kidney-Disease\.conda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [12]:
input_data = pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,42,7,84000,1,1,1,50000


In [13]:
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,42,7,84000,1,1,1,50000


In [14]:
input_data = pd.concat([input_data.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_data 

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,42,7,84000,1,1,1,50000,1.0,0.0,0.0


In [15]:
input_scaled = scaler.transform(input_data)
input_scaled

array([[-0.52046273,  0.91509065,  0.29735181,  0.68441506,  0.12083887,
        -0.91150957,  0.64376017,  0.97725852, -0.87097985,  1.00150113,
        -0.58312392, -0.57273139]])

In [16]:
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step


array([[0.06555307]], dtype=float32)

In [17]:
prediction_proba = prediction[0][0]

In [ ]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
